In [6]:
from optuna.integration import lightgbm as op_lgb
import lightgbm as lgb
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import tqdm

In [2]:
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")

train_df["flag"] , test_df["flag"] = "train", "test"
fe_df = pd.concat([train_df,test_df]).reset_index(drop=True)
#fe_df = pd.read_csv("../fe_df.csv")

In [3]:
en = LabelEncoder()
fe_df["trainNo"] = en.fit_transform(fe_df["trainNo"].values)
fe_df["stopStation"] = en.fit_transform(fe_df["stopStation"].values)
fe_df["lineName"] = en.fit_transform(fe_df["lineName"].values)

fe_df["plan_H"] = fe_df["planArrival"].apply(lambda x:int(x.split(":")[0]))
fe_df["plan_m"] = fe_df["planArrival"].apply(lambda x:int(x.split(":")[1]))

In [4]:
fe_df["linename_after_delayTime"] = 0
fe_df["linename_after_delayTime1"] = 0
fe_df["linename_after_delayTime2"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName"])):
    fe_df.loc[df.index.values,"linename_after_delayTime"] = df["delayTime"].shift().fillna(method="ffill").fillna(0)
    fe_df.loc[df.index.values,"linename_after_delayTime1"] = df["delayTime"].shift(2).fillna(method="ffill").fillna(0)
    fe_df.loc[df.index.values,"linename_after_delayTime2"] = df["delayTime"].shift(3).fillna(method="ffill").fillna(0)
    
fe_df["trainNo_after_delayTime"] = 0
fe_df["trainNo_after_delayTime1"] = 0
fe_df["trainNo_after_delayTime2"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","trainNo"])):
    fe_df.loc[df.index.values,"trainNo_after_delayTime"] = df["delayTime"].shift().fillna(method="ffill").fillna(0)
    fe_df.loc[df.index.values,"trainNo_after_delayTime1"] = df["delayTime"].shift(2).fillna(method="ffill").fillna(0)
    fe_df.loc[df.index.values,"trainNo_after_delayTime2"] = df["delayTime"].shift(3).fillna(method="ffill").fillna(0)

100%|████████████████████████████████████████████████████████████████████████| 136820/136820 [2:25:39<00:00, 15.66it/s]


In [5]:
"""
fe_df["count_lineName"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName"])):
    a = df.reset_index(drop=True)
    fe_df.loc[df.index.values,"count_lineName"] = a.index.values

fe_df["count_trainNo"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","trainNo"])):
    a = df.reset_index(drop=True)
    fe_df.loc[df.index.values,"count_trainNo"] = a.index.values

fe_df["count_stopStation"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","stopStation"])):
    a = df.reset_index()
    fe_df.loc[df.index.values,"count_stopStation"] = a.index.values

fe_df["count_lieName_trainNo"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName","trainNo"])):
    a = df.reset_index()
    fe_df.loc[df.index.values,"count_lieName_trainNo"] = a.index.values
        
fe_df["count_lieName_stopStation"] = 0
for name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName","stopStation"])):
    a = df.reset_index()
    fe_df.loc[df.index.values,"count_lieName_stopStation"] = a.index.values
        
"""

'\nfe_df["count_lineName"] = 0\nfor name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName"])):\n    a = df.reset_index(drop=True)\n    fe_df.loc[df.index.values,"count_lineName"] = a.index.values\n\nfe_df["count_trainNo"] = 0\nfor name , df in tqdm.tqdm(fe_df.groupby(by=["date","trainNo"])):\n    a = df.reset_index(drop=True)\n    fe_df.loc[df.index.values,"count_trainNo"] = a.index.values\n\nfe_df["count_stopStation"] = 0\nfor name , df in tqdm.tqdm(fe_df.groupby(by=["date","stopStation"])):\n    a = df.reset_index()\n    fe_df.loc[df.index.values,"count_stopStation"] = a.index.values\n\nfe_df["count_lieName_trainNo"] = 0\nfor name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName","trainNo"])):\n    a = df.reset_index()\n    fe_df.loc[df.index.values,"count_lieName_trainNo"] = a.index.values\n        \nfe_df["count_lieName_stopStation"] = 0\nfor name , df in tqdm.tqdm(fe_df.groupby(by=["date","lineName","stopStation"])):\n    a = df.reset_index()\n    fe_df.loc[df.index.values

In [6]:
#fe_df = fe_df.drop(["count_trainNo_stopStation"],axis=1)

In [7]:
fe_df.to_csv("../fe_df.csv",index=None)

In [2]:
fe_df = pd.read_csv("../fe_df.csv")

In [3]:
fe_df

,id,date,lineName,directionCode,trainNo,stopStation,planArrival,delayTime,flag,target,plan_H,plan_m,linename_after_delayTime,linename_after_delayTime1,linename_after_delayTime2,trainNo_after_delayTime,trainNo_after_delayTime1,trainNo_after_delayTime2
0,0,20191204,0,1,11,37,06:11,0.0,train,NaN,6,11,0.0,0.0,0.0,0.0,0.0,0.0
1,1,20191204,0,1,11,53,06:13,0.0,train,NaN,6,13,0.0,0.0,0.0,0.0,0.0,0.0
2,2,20191204,0,1,11,22,06:16,0.0,train,NaN,6,16,0.0,0.0,0.0,0.0,0.0,0.0
3,3,20191204,0,1,11,44,06:18,0.0,train,NaN,6,18,0.0,0.0,0.0,0.0,0.0,0.0
4,4,20191204,0,1,11,72,06:20,0.0,train,NaN,6,20,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748109,2748109,20201021,3,2,2969,3,18:47,0.0,test,0.0,18,47,0.0,0.0,0.0,0.0,0.0,0.0
2748110,2748110,20201021,3,2,2969,2,18:51,0.0,test,0.0,18,51,0.0,0.0,0.0,0.0,0.0,0.0
2748111,2748111,20201021,3,2,2969,95,18:53,0.0,test,0.0,18,53,0.0,0.0,0.0,0.0,0.0,0.0
2748112,2748112,20201021,3,2,2969,101,18:59,0.0,test,0.0,18,59,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
models = [{'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 1.8733056155012844e-08,
  'lambda_l2': 0.042175441158621894,
  'num_leaves': 253,
  'feature_fraction': 1.0,
  'bagging_fraction': 0.9432290226490325,
  'bagging_freq': 1,
  'min_child_samples': 5,
  'num_iterations':1000},
 {'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 0.0,
  'lambda_l2': 0.0,
  'num_leaves': 254,
  'feature_fraction': 1.0,
  'bagging_fraction': 0.9496759825203145,
  'bagging_freq': 1,
  'min_child_samples': 5,
 'num_iterations':1000},
 {'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 2.3170289027538067e-07,
  'lambda_l2': 2.4847712519032745,
  'num_leaves': 237,
  'feature_fraction': 0.92,
  'bagging_fraction': 0.985015085793738,
  'bagging_freq': 1,
  'min_child_samples': 5,
 'num_iterations':1000},
 {'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 4.513441354041495e-08,
  'lambda_l2': 1.1107266816796869e-07,
  'num_leaves': 255,
  'feature_fraction': 0.8999999999999999,
  'bagging_fraction': 1.0,
  'bagging_freq': 0,
  'min_child_samples': 5,
 'num_iterations':1000}]

In [22]:
k = 4
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2021)

fe = fe_df.drop(["id","date","flag","target","delayTime","planArrival"],axis=1).columns
train = fe_df[fe_df["flag"]=="train"]
test = fe_df[fe_df["flag"]=="test"]
test = test[test["target"]==1.0]
x = train[fe]
y = train["delayTime"]
"""
lgbm_params = {'objective': 'regression',
               'random_state': 2021,
               'metric': 'mae',
               'verbosity': -1,
               'learning_rate': 0.01,
               #'drop_rate': 0.3,
               'gpu_use_dp':True,
               "early_stopping_rounds":60}
"""
model_params = []
model_score = []
best_iteration = []

test["delayTime"] = 0
pred = test["delayTime"].values

model_count = 0
for train_idx , val_idx in skf.split(x,y):
    train_x = x.iloc[train_idx]
    train_y = y.iloc[train_idx]
    val_x = x.iloc[val_idx]
    val_y = y.iloc[val_idx]
    
    train_set = lgb.Dataset(train_x,train_y)
    val_set = lgb.Dataset(val_x, val_y, reference=train_set)
    
    lgbm_params = models[model_count]
    model = lgb.train(lgbm_params,
                      train_set,
                      valid_sets=val_set)
    model_count += 1
    model_params.append(model.params)
    model_score.append(model.best_score)
    best_iteration.append(model.best_iteration)
    
    pred_gbm = model.predict(test[fe],num_iteration=model.best_iteration)
    pred = pred + pred_gbm
pred = pred/k

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(("The least populated class in y has only %d"
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 1.90941
[2]	valid_0's l1: 1.8917
[3]	valid_0's l1: 1.87416
[4]	valid_0's l1: 1.85681
[5]	valid_0's l1: 1.83962
[6]	valid_0's l1: 1.82261
[7]	valid_0's l1: 1.80577
[8]	valid_0's l1: 1.78911
[9]	valid_0's l1: 1.7726
[10]	valid_0's l1: 1.75627
[11]	valid_0's l1: 1.7401
[12]	valid_0's l1: 1.7241
[13]	valid_0's l1: 1.70826
[14]	valid_0's l1: 1.69257
[15]	valid_0's l1: 1.67703
[16]	valid_0's l1: 1.66167
[17]	valid_0's l1: 1.64644
[18]	valid_0's l1: 1.63135
[19]	valid_0's l1: 1.61643
[20]	valid_0's l1: 1.60169
[21]	valid_0's l1: 1.58719
[22]	valid_0's l1: 1.57291
[23]	valid_0's l1: 1.55879
[24]	valid_0's l1: 1.54484
[25]	valid_0's l1: 1.531
[26]	valid_0's l1: 1.51733
[27]	valid_0's l1: 1.50376
[28]	valid_0's l1: 1.49034
[29]	valid_0's l1: 1.47708
[30]	valid_0's l1: 1.46393
[31]	valid_0's l1: 1.45094
[32]	valid_0's l1: 1.4381
[33]	valid_0's l1: 1.42538
[34]	valid_0's l1: 1.41279
[35]	valid_0's l1: 1.40035
[36]	valid_0's l1: 1.38801
[37]	valid_0's l1: 1.37582
[38]	valid_0's l1

[293]	valid_0's l1: 0.307338
[294]	valid_0's l1: 0.306612
[295]	valid_0's l1: 0.30589
[296]	valid_0's l1: 0.305168
[297]	valid_0's l1: 0.304476
[298]	valid_0's l1: 0.303783
[299]	valid_0's l1: 0.303096
[300]	valid_0's l1: 0.302397
[301]	valid_0's l1: 0.301732
[302]	valid_0's l1: 0.301074
[303]	valid_0's l1: 0.300428
[304]	valid_0's l1: 0.299781
[305]	valid_0's l1: 0.299132
[306]	valid_0's l1: 0.298502
[307]	valid_0's l1: 0.297868
[308]	valid_0's l1: 0.297242
[309]	valid_0's l1: 0.296624
[310]	valid_0's l1: 0.296029
[311]	valid_0's l1: 0.295426
[312]	valid_0's l1: 0.2948
[313]	valid_0's l1: 0.294221
[314]	valid_0's l1: 0.293645
[315]	valid_0's l1: 0.293075
[316]	valid_0's l1: 0.292521
[317]	valid_0's l1: 0.291961
[318]	valid_0's l1: 0.291391
[319]	valid_0's l1: 0.290844
[320]	valid_0's l1: 0.290293
[321]	valid_0's l1: 0.289751
[322]	valid_0's l1: 0.289224
[323]	valid_0's l1: 0.288704
[324]	valid_0's l1: 0.288169
[325]	valid_0's l1: 0.287633
[326]	valid_0's l1: 0.287117
[327]	valid_0's l

[581]	valid_0's l1: 0.241154
[582]	valid_0's l1: 0.241117
[583]	valid_0's l1: 0.241076
[584]	valid_0's l1: 0.241032
[585]	valid_0's l1: 0.240993
[586]	valid_0's l1: 0.240952
[587]	valid_0's l1: 0.240909
[588]	valid_0's l1: 0.240883
[589]	valid_0's l1: 0.24084
[590]	valid_0's l1: 0.240814
[591]	valid_0's l1: 0.240769
[592]	valid_0's l1: 0.240722
[593]	valid_0's l1: 0.24068
[594]	valid_0's l1: 0.240652
[595]	valid_0's l1: 0.240625
[596]	valid_0's l1: 0.24059
[597]	valid_0's l1: 0.240556
[598]	valid_0's l1: 0.240513
[599]	valid_0's l1: 0.240472
[600]	valid_0's l1: 0.240428
[601]	valid_0's l1: 0.240388
[602]	valid_0's l1: 0.240352
[603]	valid_0's l1: 0.240323
[604]	valid_0's l1: 0.240289
[605]	valid_0's l1: 0.240262
[606]	valid_0's l1: 0.240242
[607]	valid_0's l1: 0.24021
[608]	valid_0's l1: 0.240181
[609]	valid_0's l1: 0.240163
[610]	valid_0's l1: 0.240119
[611]	valid_0's l1: 0.240071
[612]	valid_0's l1: 0.240045
[613]	valid_0's l1: 0.240017
[614]	valid_0's l1: 0.239994
[615]	valid_0's l1

[867]	valid_0's l1: 0.235822
[868]	valid_0's l1: 0.235809
[869]	valid_0's l1: 0.235809
[870]	valid_0's l1: 0.235794
[871]	valid_0's l1: 0.235788
[872]	valid_0's l1: 0.235789
[873]	valid_0's l1: 0.235783
[874]	valid_0's l1: 0.235779
[875]	valid_0's l1: 0.235768
[876]	valid_0's l1: 0.235759
[877]	valid_0's l1: 0.235754
[878]	valid_0's l1: 0.235752
[879]	valid_0's l1: 0.235731
[880]	valid_0's l1: 0.235717
[881]	valid_0's l1: 0.235711
[882]	valid_0's l1: 0.235705
[883]	valid_0's l1: 0.235698
[884]	valid_0's l1: 0.235692
[885]	valid_0's l1: 0.235689
[886]	valid_0's l1: 0.235675
[887]	valid_0's l1: 0.235672
[888]	valid_0's l1: 0.235672
[889]	valid_0's l1: 0.235671
[890]	valid_0's l1: 0.235669
[891]	valid_0's l1: 0.235663
[892]	valid_0's l1: 0.235654
[893]	valid_0's l1: 0.235641
[894]	valid_0's l1: 0.235639
[895]	valid_0's l1: 0.235634
[896]	valid_0's l1: 0.235624
[897]	valid_0's l1: 0.235614
[898]	valid_0's l1: 0.235613
[899]	valid_0's l1: 0.235605
[900]	valid_0's l1: 0.2356
[901]	valid_0's 

...

[592]	valid_0's l1: 0.24114
[593]	valid_0's l1: 0.241111
[594]	valid_0's l1: 0.241075
[595]	valid_0's l1: 0.241056
[596]	valid_0's l1: 0.241016
[597]	valid_0's l1: 0.240975
[598]	valid_0's l1: 0.240943
[599]	valid_0's l1: 0.240926
[600]	valid_0's l1: 0.240888
[601]	valid_0's l1: 0.240846
[602]	valid_0's l1: 0.240801
[603]	valid_0's l1: 0.240747
[604]	valid_0's l1: 0.240695
[605]	valid_0's l1: 0.240676
[606]	valid_0's l1: 0.240645
[607]	valid_0's l1: 0.240621
[608]	valid_0's l1: 0.240597
[609]	valid_0's l1: 0.240549
[610]	valid_0's l1: 0.240507
[611]	valid_0's l1: 0.240478
[612]	valid_0's l1: 0.240443
[613]	valid_0's l1: 0.240406
[614]	valid_0's l1: 0.240359
[615]	valid_0's l1: 0.240328
[616]	valid_0's l1: 0.240294
[617]	valid_0's l1: 0.240265
[618]	valid_0's l1: 0.240238
[619]	valid_0's l1: 0.240201
[620]	valid_0's l1: 0.240155
[621]	valid_0's l1: 0.240133
[622]	valid_0's l1: 0.240114
[623]	valid_0's l1: 0.240084
[624]	valid_0's l1: 0.240043
[625]	valid_0's l1: 0.24001
[626]	valid_0's 

[877]	valid_0's l1: 0.235625
[878]	valid_0's l1: 0.235616
[879]	valid_0's l1: 0.235611
[880]	valid_0's l1: 0.235607
[881]	valid_0's l1: 0.235592
[882]	valid_0's l1: 0.235576
[883]	valid_0's l1: 0.23557
[884]	valid_0's l1: 0.235559
[885]	valid_0's l1: 0.235559
[886]	valid_0's l1: 0.235551
[887]	valid_0's l1: 0.235546
[888]	valid_0's l1: 0.235535
[889]	valid_0's l1: 0.235534
[890]	valid_0's l1: 0.235527
[891]	valid_0's l1: 0.235522
[892]	valid_0's l1: 0.235521
[893]	valid_0's l1: 0.235504
[894]	valid_0's l1: 0.2355
[895]	valid_0's l1: 0.235493
[896]	valid_0's l1: 0.235485
[897]	valid_0's l1: 0.235481
[898]	valid_0's l1: 0.235469
[899]	valid_0's l1: 0.235463
[900]	valid_0's l1: 0.235461
[901]	valid_0's l1: 0.235449
[902]	valid_0's l1: 0.235444
[903]	valid_0's l1: 0.235437
[904]	valid_0's l1: 0.235435
[905]	valid_0's l1: 0.235421
[906]	valid_0's l1: 0.235409
[907]	valid_0's l1: 0.235392
[908]	valid_0's l1: 0.235387
[909]	valid_0's l1: 0.23538
[910]	valid_0's l1: 0.235376
[911]	valid_0's l1

[168]	valid_0's l1: 0.511182
[169]	valid_0's l1: 0.508169
[170]	valid_0's l1: 0.505178
[171]	valid_0's l1: 0.502228
[172]	valid_0's l1: 0.499322
[173]	valid_0's l1: 0.496455
[174]	valid_0's l1: 0.493609
[175]	valid_0's l1: 0.490793
[176]	valid_0's l1: 0.488002
[177]	valid_0's l1: 0.485249
[178]	valid_0's l1: 0.482533
[179]	valid_0's l1: 0.479849
[180]	valid_0's l1: 0.477184
[181]	valid_0's l1: 0.474548
[182]	valid_0's l1: 0.471909
[183]	valid_0's l1: 0.469337
[184]	valid_0's l1: 0.466799
[185]	valid_0's l1: 0.464282
[186]	valid_0's l1: 0.461767
[187]	valid_0's l1: 0.459318
[188]	valid_0's l1: 0.456886
[189]	valid_0's l1: 0.454478
[190]	valid_0's l1: 0.452098
[191]	valid_0's l1: 0.449744
[192]	valid_0's l1: 0.447402
[193]	valid_0's l1: 0.445087
[194]	valid_0's l1: 0.442786
[195]	valid_0's l1: 0.440521
[196]	valid_0's l1: 0.438279
[197]	valid_0's l1: 0.436076
[198]	valid_0's l1: 0.433885
[199]	valid_0's l1: 0.43173
[200]	valid_0's l1: 0.429606
[201]	valid_0's l1: 0.427498
[202]	valid_0's

[455]	valid_0's l1: 0.250372
[456]	valid_0's l1: 0.250264
[457]	valid_0's l1: 0.250119
[458]	valid_0's l1: 0.249979
[459]	valid_0's l1: 0.249851
[460]	valid_0's l1: 0.249729
[461]	valid_0's l1: 0.249612
[462]	valid_0's l1: 0.249474
[463]	valid_0's l1: 0.249343
[464]	valid_0's l1: 0.24924
[465]	valid_0's l1: 0.249126
[466]	valid_0's l1: 0.249008
[467]	valid_0's l1: 0.248901
[468]	valid_0's l1: 0.248778
[469]	valid_0's l1: 0.248668
[470]	valid_0's l1: 0.248555
[471]	valid_0's l1: 0.248438
[472]	valid_0's l1: 0.248319
[473]	valid_0's l1: 0.248203
[474]	valid_0's l1: 0.248084
[475]	valid_0's l1: 0.247989
[476]	valid_0's l1: 0.247851
[477]	valid_0's l1: 0.24774
[478]	valid_0's l1: 0.247619
[479]	valid_0's l1: 0.247515
[480]	valid_0's l1: 0.247409
[481]	valid_0's l1: 0.247282
[482]	valid_0's l1: 0.247184
[483]	valid_0's l1: 0.247085
[484]	valid_0's l1: 0.246984
[485]	valid_0's l1: 0.246899
[486]	valid_0's l1: 0.24681
[487]	valid_0's l1: 0.246716
[488]	valid_0's l1: 0.246624
[489]	valid_0's l

[742]	valid_0's l1: 0.237034
[743]	valid_0's l1: 0.237013
[744]	valid_0's l1: 0.237
[745]	valid_0's l1: 0.236979
[746]	valid_0's l1: 0.23697
[747]	valid_0's l1: 0.236958
[748]	valid_0's l1: 0.236931
[749]	valid_0's l1: 0.236904
[750]	valid_0's l1: 0.236884
[751]	valid_0's l1: 0.236863
[752]	valid_0's l1: 0.236839
[753]	valid_0's l1: 0.236809
[754]	valid_0's l1: 0.236785
[755]	valid_0's l1: 0.236765
[756]	valid_0's l1: 0.236747
[757]	valid_0's l1: 0.23672
[758]	valid_0's l1: 0.236697
[759]	valid_0's l1: 0.236683
[760]	valid_0's l1: 0.23666
[761]	valid_0's l1: 0.236635
[762]	valid_0's l1: 0.236619
[763]	valid_0's l1: 0.236602
[764]	valid_0's l1: 0.23659
[765]	valid_0's l1: 0.236584
[766]	valid_0's l1: 0.236566
[767]	valid_0's l1: 0.236558
[768]	valid_0's l1: 0.236533
[769]	valid_0's l1: 0.236509
[770]	valid_0's l1: 0.236492
[771]	valid_0's l1: 0.23648
[772]	valid_0's l1: 0.236466
[773]	valid_0's l1: 0.236456
[774]	valid_0's l1: 0.236439
[775]	valid_0's l1: 0.236421
[776]	valid_0's l1: 0.

In [23]:
model_score

[defaultdict(collections.OrderedDict,
             {'valid_0': OrderedDict([('l1', 0.2350028563155758)])}),
 defaultdict(collections.OrderedDict,
             {'valid_0': OrderedDict([('l1', 0.23342052383885833)])}),
 defaultdict(collections.OrderedDict,
             {'valid_0': OrderedDict([('l1', 0.23487373143330645)])}),
 defaultdict(collections.OrderedDict,
             {'valid_0': OrderedDict([('l1', 0.2346397568657935)])})]

In [24]:
model_params

[{'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 1.8733056155012844e-08,
  'lambda_l2': 0.042175441158621894,
  'num_leaves': 253,
  'feature_fraction': 1.0,
  'bagging_fraction': 0.9432290226490325,
  'bagging_freq': 1,
  'min_child_samples': 5},
 {'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 0.0,
  'lambda_l2': 0.0,
  'num_leaves': 254,
  'feature_fraction': 1.0,
  'bagging_fraction': 0.9496759825203145,
  'bagging_freq': 1,
  'min_child_samples': 5},
 {'objective': 'regression',
  'random_state': 2021,
  'metric': 'l1',
  'verbosity': -1,
  'learning_rate': 0.01,
  'gpu_use_dp': True,
  'feature_pre_filter': False,
  'lambda_l1': 2.3170289027538067e-07,
  'lambda_l2': 2.4847712519032745,
  'num_leaves': 237,
  'featur

In [25]:
best_iteration

[0, 0, 0, 0]

In [26]:
submit = pd.read_csv("../sample_submit.csv",header=None,names=["id","pred"])
submit["pred"] = pred
submit["pred"] = submit["pred"].apply(lambda x:100 if x>100 else x)
submit.to_csv("../output/model1.csv",header=None,index=False)

In [27]:
submit

,id,pred
0,1488935,2.772759
1,1488936,2.869095
2,1488937,2.996803
3,1488938,2.876329
4,1488939,2.910473
...,...,...
123270,2748010,1.252080
123271,2748011,1.077471
123272,2748012,1.114386
123273,2748013,1.250966


In [28]:
submit = pd.read_csv("../output/submit_lgb_model.csv",header=None)
submit

,0,1
0,1488935,2.772759
1,1488936,2.869095
2,1488937,2.996803
3,1488938,2.876329
4,1488939,2.910473
...,...,...
123270,2748010,1.252080
123271,2748011,1.077471
123272,2748012,1.114386
123273,2748013,1.250966


In [29]:
submit = pd.read_csv("../sample_submit.csv",header=None,names=["id","pred"])
submit["pred"] = pred
submit["pred"] = submit["pred"].apply(lambda x:100 if x>100 else x).apply(lambda x:np.round(x))
submit.to_csv("../output/model1_round.csv",header=None,index=False)